## kNN classifier

In [81]:
import pandas as pd
import matplotlib.pyplot as plt
from numpy  import *
url="https://coding.net/u/HongHuangNeu/p/Machine-Learning-Notes-Data/git/raw/master/DatingData/datingTestSet2.txt"
df=pd.read_csv(url,sep='\t')
df[:3]

,fre_flier_miles_earned_per_year,per_of_time_spent_playing_video_games,liters_of_ice_cream_consumed_per_year,label
0,40920,8.326976,0.953952,3
1,14488,7.153469,1.673904,2
2,26052,1.441871,0.805124,1


Let's visualize the data set. First

In [82]:
indexed_df = df.set_index(['label'])
indexed_df[:3]

,fre_flier_miles_earned_per_year,per_of_time_spent_playing_video_games,liters_of_ice_cream_consumed_per_year
label,,,
3,40920,8.326976,0.953952
2,14488,7.153469,1.673904
1,26052,1.441871,0.805124


In [83]:
#cmap = {1.0: 'red', 2.0: 'blue', 3.0: 'yellow'}

#df.plot(x='liters_of_ice_cream_consumed_per_year', y='per_of_time_spent_playing_video_games', kind='scatter', c=[cmap.get(c, 'black') for c in df.label])

#plt.show()

In [84]:
matrix=df.values
labels=matrix[:,3]   #the 3rd column is the label column
features=matrix[:,0:3] #the 0th to 2nd column is the feature columns


### Define the classification function

In [85]:
def classify(x,trainingSet,labels,k):
    numberOfRows=trainingSet.shape[0];
    # repeat x for numberOfRows times 
    xs=tile(x,(numberOfRows,1))
    #calculate the difference of coordinates for each dimension
    diff=trainingSet-xs
    #calculate the square of the differences
    square=diff*diff
    #calculate distances
    distances=(square.sum(axis=1))**0.5
    
    #sort the distances
    sortedIndexes=distances.argsort()
    #calculate the frequencies of classes in top k
    classCount={}
    for i in range(k):
        label=labels[sortedIndexes[i]]
        if label in classCount:
            classCount[label]=classCount[label]+1
        else:
            classCount[label]=1
    #sort the resulted map by value
    sortedClassLabels=sorted(classCount.items(),key=lambda x: x[1],reverse=True)
    return sortedClassLabels[0][0]


    

Let's play with this classifier first

In [86]:
#play with this classifier first
x=array([1,2,3]) #vector to classify
classify(x,features,labels,3)

2.0

## Normalization

$$newValue=\frac{oldValue-minValue}{maxValue-minValue}$$

In [87]:
# this function process training set
def norm(dataSet):
    minValue=dataSet.min(axis=0)
    maxValue=dataSet.max(axis=0)
    numberOfRows=dataSet.shape[0]
    minValues=tile(minValue,(numberOfRows,1))
    maxValues=tile(maxValue,(numberOfRows,1))
    diff=maxValues-minValues
    normedDataSet=zeros(shape(dataSet))
    normedDataSet=(dataSet-minValues)/diff
    return normedDataSet,minValue,maxValue

#this function process the data point to classify
#it takes the min values and max values calculated from the training set
def getValue(testPoint,minValue,maxValue):
    normedTestSet=zeros(shape(testPoint))
    diff=maxValue-minValue
    normedTestPoint=(testPoint-minValue)/diff
    return normedTestPoint
    
    

call this norm function to normalize the data point and do the classification

In [88]:
normedTrainSet,minValue,maxValue=norm(features)
newX=getValue(x,minValue,maxValue)
classify(newX,normedTrainSet,labels,3)


2.0

## Test the classifier
Use part of the training data set as test set

In [89]:
def testClassifier(ratio,dataSet,labels):
    normalizedDataSet,minValue,maxValue=norm(dataSet)
    numberOfRows=dataSet.shape[0]
    numberOfTestPoints=int(numberOfRows*ratio)
    totalCount=0
    errorCount=0
    for i in range(numberOfTestPoints):
        x=normalizedDataSet[i]
        label=classify(x,normalizedDataSet[numberOfTestPoints:numberOfRows],labels[numberOfTestPoints:numberOfRows],3)
        totalCount=totalCount+1
        if(label!=labels[i]):
            errorCount=errorCount+1
        
    errorRate=errorCount/totalCount
    return errorRate



Use to 10% of the data points as test data set to test the classifier

In [90]:
testClassifier(0.1,features,labels)

0.05